# California Housing
## Overview

When looking through datasets that contain over 20,000 instances of housing data, it would be very unlikely to not wonder about the insights that the data may possibly hold.

One may ask:
- What if you wanted to get rich off of real estate?
- What if you wanted to predict the next housing bubble?
- What if you could produce invaluable insights that could be the basis for further research?

Although the first two out of the three speculations are unlikely to result from just reading into our two datasets, it would be a waste to not break these chunks of information apart and see what we can get, but before we delve deep into our datasets, we should consider the limitations of our data.

1. The Kaggle dataset ("housing.csv") is an extracted (but not cleaned) version of the California 1990 Census and it holds median and total numerical values for housing features per district, separated by latitudinal and longitudinal coordinates. Kaggle is an online community platform for data enthusiasts. This dataset is a modified version from data that was gathered by associate professor, Luis Torgo, from the University of Porto, whom of which had collected the data from the California Census of 1990.

2. The recent housing listings data ("Total.csv"), which were posted from up to a year since 19 September 2018, only shows listings that are posted by real estate brokers on an industry-accepted information dissemination system, the California Regional Multiple Listing Service (CRMLS); therefore, off-market listings <u>are not</u> represented in the data.

Although there are some lingering questions that lay underneath our data (considering how it was obtained) and our resources are limited as to how accurately we can produce our analyses, we still are tasked to come to a data-driven conclusion to find out the answer to the following question:

If we took the top and bottom five percentiles of houses in California during the 1990s, in terms of median house value, and compared them to houses of the same standing that have been listed for sale within the last year in the same market, would there be a statistical difference between the two groups?

## Data Exploration and Cleanup

### Setting up and getting our hands dirty with the data
Before we can extract any meaningful and accurate analyses from our data, we must first make an attempt to understand all of its facets and not just search for whatever we want; that means, we must first 'get our hands dirty' with it. Although it can be irresistably fun just to mess around with our data, we should be aware that there is a more valuable incentive for doing so, which is described by the XY Problem.

** For further reasoning, please refer to the following link to a discussion which details the XY Problem: https://meta.stackexchange.com/questions/66377/what-is-the-xy-problem*

In order to be able to easily work with our data, we must first import the necessary dependencies and store our data.

In [ ]:
# Import dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import gmplot
import scipy as stats
from config import api_key

In [ ]:
% matplotlib inline

In [ ]:
# Read/store data
ca_1990 = pd.read_csv('Data/housing.csv')
ca_current = pd.read_csv('Data/Total.csv')

#### How many total instances are there?

In [ ]:
ca_1990.count()

#### Let's drop the rows with any missing values

In [ ]:
ca_1990 = ca_1990.dropna()

In [ ]:
ca_1990.count()

#### Reason for dropping values
There is an approximate 1% difference when rows with any empty values are dropped and the data is derived as the median of each of its categories per the district; therefore, the values are negligible.

### California Housing (1990) Analysis

#### The Search for Relationships
We were looking at the several features that was listed within our data (such as the longitudinal and latitudinal coordinates of each district, the median ages of the houses, the total number of rooms, and so forth)  and made several charts in an attempt to draw insights which may be lying underneath, and they are as follows:

#### Total Rooms per District vs Total Population District

In [ ]:
bins = [0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000]
group_names = ["<5k", "5k-10k", "10k-15k", "15k-20k", "20k-25k", "25k-30k", "30k-35k", ">35k"]
ca_1990["total_rooms_groups"] = pd.cut(ca_1990['total_rooms'], bins, labels=group_names)

plt.scatter(ca_1990['total_rooms_groups'], ca_1990['population'])
plt.xlabel('Total Rooms')
plt.ylabel('Population')
plt.title('Total Rooms per District vs Population per District')
plt.grid()
plt.savefig("pngs/pop_vs_rooms_sub_groups.png", dpi=300)
plt.show()

* *The chart above depicts the frequencies for several ranges of number of total rooms per the district and is set against the total population of a district.*
* *It is intriguing to see that there is a concentration of districts that have zero to twenty thousand rooms and populations from zero to thirteen thousand.*

#### Top Five Percent of Median House Values vs Total Population

In [ ]:
# Slice the top 5%
top5 = ca_1990.iloc[:round(len(ca_1990['median_house_value'])*0.05), :].sort_values('median_house_value', ascending=False)
top5.head()

In [ ]:
bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 13000]
group_names = ["<1k", "1k-2k", "2k-3k", "3k-4k", "4k-5k", "5k-6k", "6k-7k", ">7k"]
top5['population_groups'] = pd.cut(top5['population'], bins, labels=group_names)
top5.head()

In [ ]:
plt.scatter(top5['median_house_value'], top5['population_groups'], c='r')
plt.xlabel('House Value')
plt.ylabel('Population')
plt.title('Total Population vs Median House Value')
plt.grid()
plt.savefig("pngs/Top5_PHV.png", dpi=300)
plt.show()

* *From the plot above, we can see that there is a general concentration in districts with populations ranging from zero to two thousand people per district, in respect to houses that rank in the top five percentile in terms of median house value.*
* *We can infer from this data that districts with this general range of population is ubiquitous and non-dependent of median house value.*
* *Intriguingly, we can see that houses that are worth approximately five hundred thousand dollars can vary from small to large numbers of population.*

#### Bottom Five Percent of Median House Values vs Total Population

In [ ]:
# Slice the bottom 5%
bottom5 = ca_1990.iloc[round(len(ca_1990['median_house_value'])*0.95): len(ca_1990['median_house_value']),:].sort_values('median_house_value', ascending=False)
bottom5.head()

In [ ]:
bins = [0, 3000, 6000, 9000, 12000, 15000, 18000]
group_names = ["<3k", "3k-6k", "6k-9k", "9k-12k", "12k-15k", ">15k"]
bottom5["population_groups"] = pd.cut(bottom5['population'], bins, labels=group_names)
plt.scatter(bottom5['median_house_value'], bottom5['population_groups'], c='g')
plt.xlabel('House Value')
plt.ylabel('Population')
plt.title('Total Population vs Median House Value')
plt.grid()
plt.savefig('pngs/Bottom5_PHV.png', dpi=300, bbox_inches='tight')
plt.show()

* *From the plot above, we can see that there is a general concentration in districts with populations ranging from zero to three thousand people per district and from three to six thousand people per district, in respect to houses that rank in the bottom five percentile in terms of median house value.*
* *We can infer from this data that districts with these general ranges of populations are ubiquitous and non-dependent of median house value.*

#### Ocean Proximity

In [ ]:
# Sort data by house value
ca_1990 = ca_1990.sort_values('median_house_value', ascending=False).reset_index(drop=True)
ca_1990.head()

#### Ocean Proximity vs Count of Houses

In [ ]:
x_axis_op = ["<1H OCEAN", "INLAND", "ISLAND", "NEAR BAY", "NEAR OCEAN"]
y_axis_op = ca_1990.groupby("ocean_proximity").count().rename(columns={"longitude": "count_of_houses"})["count_of_houses"]
plt.bar(x_axis_op, y_axis_op, alpha=0.5, align='center')
plt.xlabel("Ocean Proximity")
plt.ylabel("Count of Houses")
op_bar_chart = plt.title("Total Number of Houses in CA per OP Category")
plt.savefig('pngs/op_vs_count_of_houses.png', dpi=300)
plt.show()

* *The bar chart above depicts the total count of houses per each category.*
* *We can see that most of the houses in California are either less than one-hour away from the ocean or are inland.*
* *The chart also communicates that the majority of Californians lived less than one hour from the ocean during the 1990s.*

In [ ]:
x_axis_op = ["<1H OCEAN", "INLAND", "NEAR BAY"]
y_axis_op = top5.groupby("ocean_proximity").count().rename(columns={"longitude": "count_of_houses"})["count_of_houses"]
plt.bar(x_axis_op, y_axis_op, color='b', alpha=0.5, align='center')
plt.xlabel("Ocean Proximity")
plt.ylabel("Count of Houses")
op_bar_chart = plt.title("Top Five Percent")
plt.savefig("pngs/top_five_op_vs_COH.png", dpi=300)
plt.show()

#### Top 5 Percent Ocean Proximity vs Count of Houses

* *The chart above shows the total count of houses per ocean proximity category for houses in the top five percent, in terms of median house value.*
* *The majority of these houses are in closer proximity to some body of water than houses that are inland.*

#### Bottom 5 Percent Ocean Proximity vs Count of Houses

In [ ]:
x_axis_op = ["<1H OCEAN", "INLAND", "NEAR OCEAN"]
y_axis_op = bottom5.groupby("ocean_proximity").count().rename(columns={"longitude": "count_of_houses"})["count_of_houses"]
plt.bar(x_axis_op, y_axis_op, color='g', alpha=0.5, align='center')
plt.xlabel("Ocean Proximity")
plt.ylabel("Count of Houses")
op_bar_chart = plt.title("Bottom Five Percent")
plt.savefig("pngs/bottom_five_op_vs_COH.png", dpi=300)
plt.show()

* *The chart above depicts the total count of houses per ocean proximity category for houses in the bottom five percent, in terms of median house value.*
* *The majority of these houses are inland; furthest from any body of water in comparison to their counterparts.*


#### Top Five Ocean Proximity Average Median Price

In [ ]:
op_mean_house_value_top = top5.groupby("ocean_proximity")["median_house_value"].mean().round(2)
op_mean_df_top = pd.DataFrame(op_mean_house_value_top)
op_mean_df_top = op_mean_df_top.rename(columns={"median_house_value":"Average Median Price"})
op_mean_df_top['Average Median Price'] = op_mean_df_top['Average Median Price'].map('${:,.2f}'.format)
op_mean_df_top

* *Although we expect that being located closer to bodies of water usually translates to a higher price per house, the averages of the median values for our top five percent of houses in California have similar prices.*
* *Intriguingly, the inland category has an average value that is higher than any of the other categories, which are all nearer to some body of water.*

#### Bottom Five Ocean Proximity Average Median Price

In [ ]:
op_mean_house_value_bottom = bottom5.groupby("ocean_proximity")["median_house_value"].mean().round(2)
op_mean_df_bottom = pd.DataFrame(op_mean_house_value_bottom)
op_mean_df_bottom = op_mean_df_bottom.rename(columns={"median_house_value":"Average Median Price"})
op_mean_df_bottom['Average Median Price'] = op_mean_df_bottom['Average Median Price'].map('${:,.2f}'.format)
op_mean_df_bottom

* *Again, we expect that being located closer to bodies of water would translate to a higher price per house, the averages of the median values for our bottom five percent of houses in California deviates from our assumptions.*
* *The same surprise is evident in this data, when compared to its opposite counterpart.*

### California Housing (2018) Analysis

In [ ]:
ca_current.head()

In [ ]:
# Keep only the columns that are useful 
ca_current = ca_current[['Sub Type', 'St#', 'St Name', 'City', 'L/C Price', 'Br/Ba', 'YrBuilt']]
ca_current.head()

* *We cleaned our data frame of any superfluous information, as either some are not quantifiable or we simply cannot work with them due to our limited sets of data.*

#### Housing Age

Before calculating for the age, we must check for any missing values that lay in our YrBuilt column and the count of all instances in our data set.

In [ ]:
count = 0
for i in ca_current['YrBuilt'].isna():
    if i == True:
        count += 1
count

In [ ]:
ca_current.count()

In [ ]:
ca_current.count().min()

We also must calculate the number of values dropped if we were to delete all rows with any missing value.

In [ ]:
ca_current.dropna().count()

* *Here, we encounter something strange. Even though our for loop above states that there are 8 NaN values underneath the YrBuilt columns, when we drop all rows with any missing value, the total number of rows dropped is only 4.*
* *If we continue to attempt with a new dataframe that utilizes the .dropna() method, we will find that there is a bug in which most values become missing; therefore, we must fill all missing values with 0 in order to solve our issue.*

In [ ]:
# Split the string to isolate the integers
yr_blt = ca_current.loc[:, 'YrBuilt'].str.split('/', expand=True)[0]
# Turn the values into a data frame
yr_blt = pd.DataFrame(yr_blt)
# Fill the empty cells with 0
yr_blt = yr_blt.fillna(0)
# Create the new columns for age
ca_current['Age'] = ''
# Calculate for age
count = 0
for i in yr_blt[0]:
    # Values that were empty are now 0 years old.
    # We will count them later to determine whether or not we will drop them.
    i = 2018 - int(i)
    ca_current['Age'][count] = i
    count += 1
ca_current.head()

* *In the data frame above, we are calculating the current ages of our houses by subtracting the current year from the year they were built.*

#### Get rid of NaN Values under YrBuilt column

In [ ]:
# Grab all values with the year 2018 since 2018 - 0 = 2018
with_nan_df = ca_current.loc[ca_current['Age'] == 2018, :]
# Grab the values that are not NaN, but built in 2018
zero_bld = with_nan_df.loc[with_nan_df['YrBuilt'] == '0/BLD', :]
zero_asr = with_nan_df.loc[with_nan_df['YrBuilt'] == '0/ASR', :]
# Remove the houses that are built in 2018 and merge the necessary data frames
without_2018_df = pd.merge(ca_current, with_nan_df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
with_zero_bld = pd.merge(without_2018_df, zero_bld, how='outer')
zero_asr['Age'] = zero_asr['Age'].apply(int)
without_nan_df = pd.merge(with_zero_bld, zero_asr, how='outer')

In [ ]:
# Create a new column with the house prices without the dollar sign 
without_nan_df['house_price'] = without_nan_df.loc[:, 'L/C Price'].str.split('$', expand=True)[1]

In [ ]:
without_nan_df['house_price'] = without_nan_df['house_price'].str.replace(',', '')

In [ ]:
# Grab the data frame and set the index to the listing price
sorted_ca_current = ca_current.set_index("L/C Price").reset_index()

Sort data by median house value and slice the top and bottom 5%

In [ ]:
# Slice the top 5%
top5_2018 = sorted_ca_current.iloc[round(len(without_nan_df['house_price'])*0.95): len(without_nan_df['house_price']),:]
top5_2018.head()

In [ ]:
# Slice the bottom 5%
bottom5_2018 = without_nan_df.iloc[:round(len(without_nan_df['house_price'])*0.05), :]
bottom5_2018.head()

## Plotting 1990 and 2018 House Data with Gmaps

In [ ]:
# Decorations
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import gmplot
from config import api_key
from pprint import pprint

In [ ]:
# Import database
df1 = pd.read_csv('Data/housing.csv')
df2 = pd.read_csv('Data/Total.csv')
df1 = df1.dropna()

### California Housing (1990) Gmap Plotting

Sort and slice data to get top and bottom 5%

In [ ]:
# SSort data by house value
df1 = df1.sort_values('median_house_value', ascending=False).reset_index(drop=True)
df1.head()

In [ ]:
# Slice the top 5%
top5 = df1.iloc[:round(len(df1['median_house_value'])*0.05), :].sort_values('median_house_value', ascending=True)

In [ ]:
# slice the bottom 5%
bottom5 = df1.iloc[round(len(df1['median_house_value'])*0.95): len(df1['median_house_value']),:].sort_values('median_house_value', ascending=True)

Plotting Top 5% and Bottom 5% on gmap (Use gmplot package)

In [ ]:
# Set the center of the map
gmap_df1 = gmplot.GoogleMapPlotter(df1['latitude'].median(),
                                   df1['longitude'].median(), 100000)
# Plot scatter points based on LatLng
gmap_df1.scatter(df1['latitude'], df1['longitude'], '#FF0000', 
                              size = 2000, marker = False ) 
# Draw out to 'Plot' folder in html format
gmap_df1.draw("gmap_df1.html")

In [ ]:
# Set the center of the map
gmap = gmplot.GoogleMapPlotter(top5['latitude'].median(),
                                   top5['longitude'].median(), 100000)
# Plot scatter points based on LatLng
gmap.scatter(top5['latitude'], top5['longitude'], '#FF0000', 
                              size = 2000, marker = False ) 
gmap.scatter(bottom5['latitude'], bottom5['longitude'], '#110870', 
                              size = 2000, marker = False ) 
# Draw out to 'Plot' folder in html format
gmap.draw("gmap.html")

### Getting Latitudinal and Longitudinal Coordinates using Geocoder API for 2018 Housing

In [ ]:
# Keep only the columns that are useful 
df2 = df2[['Sub Type', 'St#', 'St Name', 'City', 'L/C Price', 'Br/Ba', 'YrBuilt']]
df2.head()

In [ ]:
yr_blt = df2.loc[:, 'YrBuilt'].str.split('/', expand=True)[0]
yr_blt = pd.DataFrame(yr_blt)
yr_blt = yr_blt.fillna(0)

In [ ]:
df2['Age'] = ''
count = 0
for i in yr_blt[0]:
        i = 2018 - int(i)
        df2['Age'][count] = i
        count += 1
df2.head()

In [ ]:
df2['house_price'] = df2.loc[:, 'L/C Price'].str.split('$', expand=True)[1] 

In [ ]:
df2.head()

In [ ]:
df2_by_value = df2.set_index("L/C Price")
df2_by_value = df2_by_value.reset_index()
df2_by_value.head()

In [ ]:
# import CityCode
citycode_df = pd.read_csv('Data/CleanCityCode_nospace.csv')
citycode_df = citycode_df[['City', 'Code']]
citycode_df['Code'] = citycode_df['Code'].str.strip()

In [ ]:
df2_by_value['City Name'] = ""
for index, row in df2_by_value.iterrows():
    for i, r in citycode_df.iterrows():
        if r['Code'] == row['City']:
            row['City Name'] = r['City']
            if index%200 == 0:
                print(index)

In [ ]:
df2_by_value["Address"] = df2_by_value["St#"].astype(str) + " " + df2_by_value["St Name"].astype(str) + " " + df2_by_value['City Name']
df2_by_value['Lat'] = ""
df2_by_value['Lng'] = ""

In [ ]:
params = {"key": api_key}
for index, row in df2_by_value.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params['address'] = row['Address']
    geo_data = requests.get(base_url, params).json()
    try:
        df2_by_value["Lat"][index] = geo_data["results"][0]["geometry"]["location"]["lat"]
        df2_by_value["Lng"][index] = geo_data["results"][0]["geometry"]["location"]["lng"]
        if index%100 == 0:
            print(index)
    except IndexError:
        print(f"Row {index} cannot be found on gmap.")
        continue

In [ ]:
# Output to CSV
df2_by_value.to_csv("Data/df2_latlng.csv")

### California Housing (2018) Gmap Plotting

In [ ]:
df2_latlng = pd.read_csv("Data/df2_latlng.csv")

In [ ]:
df2_latlng = df2_latlng[['L/C Price','Lat', 'Lng']].dropna()

In [ ]:
top5_df2 = df2_latlng.iloc[:round(len(df2_latlng['L/C Price'])*0.05), :].sort_values('L/C Price', ascending=True)
bottom5_df2 = df2_latlng.iloc[round(len(df2_latlng['L/C Price'])*0.95): len(df2_latlng['L/C Price']),:].sort_values('L/C Price', ascending=True)

In [ ]:
# Set the center of the map
gmap_df2_tab = gmplot.GoogleMapPlotter(top5_df2['Lat'].median(),
                                   top5_df2['Lng'].median(), 100000)
# Plot scatter points based on LatLng
gmap_df2_tab.scatter(top5_df2['Lat'], top5_df2['Lng'], '#FF0000', 
                              size = 2000, marker = False ) 
gmap_df2_tab.scatter(bottom5_df2['Lat'], bottom5_df2['Lng'], '#110870', 
                              size = 2000, marker = False ) 
# Draw out to 'Plot' folder in html format
gmap_df2_tab.draw("Plot/gmap_df2_tab.html")

In [ ]:
# Set the center of the map
gmap_df2 = gmplot.GoogleMapPlotter(df2_latlng['Lat'].median(),
                                df2_latlng['Lng'].median(), 100)
# Plot scatter points based on LatLng
gmap_df2.scatter(df2_latlng['Lat'], df2_latlng['Lng'], '#FF0000', 
                              size = 1000, marker = False) 
# Draw out to 'Plot' folder in html format
gmap_df2.draw("Plot/gmap_df2.html")

## t-test

In [ ]:
# Import database
df1 = pd.read_csv('Data/housing.csv')
df2 = pd.read_csv('Data/Total.csv')
df1 = df1.dropna()

In [ ]:
# SSort data by house value
df1 = df1.sort_values('median_house_value', ascending=False).reset_index(drop=True)
df1.head()

In [ ]:
# Keep only the columns that are useful 
df2 = df2[['Sub Type', 'St#', 'St Name', 'City', 'L/C Price', 'Br/Ba', 'YrBuilt']]
df2.head()
#df2 = df2.drop(on=0)

In [ ]:
df2['house_price'] = df2.loc[:, 'L/C Price'].str.split('$', expand=True)[1] 

In [ ]:
df2.head()

In [ ]:
df2_by_value = df2.set_index("L/C Price")
df2_by_value = df2_by_value.reset_index()
df2_by_value.head()

In [ ]:
df2['L/C Price'] = df2['L/C Price'].str.strip('$')
df2['L/C Price'] = df2['L/C Price'].str.replace(',', "").astype(float)

In [ ]:
s1 = df1['median_house_value']
s2 = df2['L/C Price']

In [ ]:
data_s1 = {'s1': s1}
data_s1 = pd.DataFrame(data_s1)
data_s2 = {'s2': s2}
data_s2 = pd.DataFrame(data_s2)

In [ ]:
(t_stat, p) = stats.ttest_ind(s1, s2, equal_var=False)
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

In [ ]:
# Normalization
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled_s1 = min_max_scaler.fit_transform(data_s1)
np_scaled_s2 = min_max_scaler.fit_transform(data_s2)

In [ ]:
(t_stat, p) = stats.ttest_ind(np_scaled_s1, np_scaled_s2, equal_var=False)
print("t-statistics is {}.".format(t_stat))
print("p-value is {}.".format(p))

## Analysis and Conclusion
We performed our t-test without normalization on the 1990 data set and 2018 data set, which gave us a p-value of 2.306824488018087e-15.
This means that there’s no significant difference between the two datasets, which conveys that the distribution of the house prices of both data sets is similar. 
We then normalized the data and got a p-value of 0, which further proves that the distribution is similar.
Overall, due to the fact that there is no statistical difference between the two data sets, the same factors that affected the prices of houses in California during the year of 1990 are very likely to still be significant in determining the prices of houses in California in the present day.